In [ ]:
import requests
from tqdm import tqdm
import pandas as pd

In [4]:
api_key = 'bd9c8d6646a2274b211a70d227734493'
start_date = 1577833200 #01-01-2020
end_date = 1640991600 #01-01-2022
cities = {
    'Warszawa' : {'lat':52.23022878686064, 'lon':21.013026391458265},
    'Wilno':{'lat':54.70227152551265, 'lon':25.26745325517322},
    'Kijów':{'lat':50.457149277152354,'lon':30.56015636937755},
    'Bratysława':{'lat':48.15708350193662, 'lon':17.12440138838671},
    'Praga':{'lat':50.07201099937842, 'lon':14.441591308044634},
    'Budapeszt':{'lat':47.489101825665614, 'lon':19.087044334480346},
    'Bukareszt':{'lat':44.438316020116694, 'lon':26.079531349342343},
    'Wiedeń':{'lat':48.2138221396932, 'lon':16.37525343160893},
    'Berno':{'lat':46.95185774655687, 'lon':7.414782769048729},
    'Bruksela':{'lat':50.88630374160421, 'lon':4.36582783309767},
    'Amsterdam':{'lat':52.38233631032214, 'lon':4.889540844826782},
    'Sztokholm':{'lat':59.32965561652341, 'lon':18.06845313423811},
    'Kopenhaga':{'lat':55.67756983322892, 'lon':12.572899977086267},
    'Madryt' : {'lat':40.41865092966006, 'lon':-3.70142548811499},
    'Berlin' : {'lat':52.51936255069942, 'lon':13.404251128255233},
    'Paryż': { 'lat':48.856186736752, 'lon':2.3514525560654733},
    'Oslo' : {'lat': 59.91372477628147, 'lon':10.752146511705593},
    'Helsinki' : {'lat':60.16990656217715, 'lon':24.937682492645845},
    'Rzym':{'lat':41.88962792531848, 'lon':12.496859107499853},
    'Mińsk':{'lat':53.90446646596854, 'lon':27.5501097846051},
    'Londyn':{'lat':51.511978652372285, 'lon':-0.12151136899130237},
    'Lizbona':{'lat':38.726486880440675, 'lon':-9.148530276798201}
}

In [5]:
rows = []
for city,geo in tqdm(cities.items()):
  city_data = requests.get(f"http://api.openweathermap.org/data/2.5/air_pollution/history?lat={geo['lat']}&lon={geo['lon']}&start={start_date}&end={end_date}&appid={api_key}").json()
  for cc in city_data['list']:
    temp = cc['components']
    temp['dt'] = cc['dt']
    temp['city'] = city
    rows.append(temp)
df = pd.DataFrame(rows)
df

100%|██████████| 22/22 [00:31<00:00,  1.45s/it]


,co,no,no2,o3,so2,pm2_5,pm10,nh3,dt,city
0,347.14,0.13,21.76,22.89,9.66,18.86,22.68,0.55,1606266000,Warszawa
1,353.81,0.18,21.94,20.56,9.54,19.24,23.14,0.58,1606269600,Warszawa
2,353.81,0.22,21.76,19.13,9.42,19.26,23.18,0.60,1606273200,Warszawa
3,350.48,0.26,21.08,18.95,9.89,18.57,22.31,0.67,1606276800,Warszawa
4,350.48,0.34,21.25,18.77,14.66,18.86,22.46,1.50,1606280400,Warszawa
...,...,...,...,...,...,...,...,...,...,...
211701,400.54,0.02,29.82,33.62,5.42,16.52,24.56,1.95,1640977200,Lizbona
211702,433.92,0.03,28.79,31.47,5.42,16.77,23.57,2.09,1640980800,Lizbona
211703,453.95,0.03,25.02,31.11,5.36,16.30,22.12,2.18,1640984400,Lizbona
211704,460.62,0.03,21.59,30.04,5.01,15.67,20.67,2.12,1640988000,Lizbona


## Policzenie uśrednień godzinowych

In [ ]:
df['co'] = df['co']*0.001
df

In [ ]:
df_index = pd.DataFrame()

for cc in set(df.city):
  df_city = df.loc[df.city==cc].reset_index(drop=True)
  dict_city = df_city.to_dict('records')
  counter = 24
  for row in dict_city[24:]:
    pm2_5_i = [x['pm2_5'] for x in dict_city[counter-24:counter]]
    row['pm2_5_avg'] = sum(pm2_5_i)/len(pm2_5_i)

    pm10_i = [x['pm10'] for x in dict_city[counter-24:counter]]
    row['pm10_avg'] = sum(pm10_i)/len(pm10_i)

    nh3_i = [x['nh3'] for x in dict_city[counter-24:counter]]
    row['nh3_avg'] = sum(nh3_i)/len(nh3_i)
    
    so2_i = [x['so2'] for x in dict_city[counter-24:counter]]
    row['so2_avg'] = sum(so2_i)/len(so2_i)

    no2_i = [x['no2'] for x in dict_city[counter-24:counter]]
    row['no2_avg'] = sum(no2_i)/len(no2_i)

    row['co_max'] = max([x['co'] for x in dict_city[counter-8:counter]])

    row['o3_max'] = max([x['o3'] for x in dict_city[counter-8:counter]])

    counter+=1
  df_index = pd.concat([df_index,pd.DataFrame.from_dict(dict_city)[24:]])

## Policzenie indeksów

In [ ]:
def get_PM25_subindex(x):
    if x <= 30:
        return x * 50 / 30
    elif x <= 60:
        return 50 + (x - 30) * 50 / 30
    elif x <= 90:
        return 100 + (x - 60) * 100 / 30
    elif x <= 120:
        return 200 + (x - 90) * 100 / 30
    elif x <= 250:
        return 300 + (x - 120) * 100 / 130
    elif x > 250:
        return 400 + (x - 250) * 100 / 130
    else:
        return 0

In [ ]:
def get_PM10_subindex(x):
    if x <= 50:
        return x
    elif x <= 100:
        return x
    elif x <= 250:
        return 100 + (x - 100) * 100 / 150
    elif x <= 350:
        return 200 + (x - 250)
    elif x <= 430:
        return 300 + (x - 350) * 100 / 80
    elif x > 430:
        return 400 + (x - 430) * 100 / 80
    else:
        return 0

In [ ]:
def get_SO2_subindex(x):
    if x <= 40:
        return x * 50 / 40
    elif x <= 80:
        return 50 + (x - 40) * 50 / 40
    elif x <= 380:
        return 100 + (x - 80) * 100 / 300
    elif x <= 800:
        return 200 + (x - 380) * 100 / 420
    elif x <= 1600:
        return 300 + (x - 800) * 100 / 800
    elif x > 1600:
        return 400 + (x - 1600) * 100 / 800
    else:
        return 0

In [ ]:
def get_NO2_subindex(x):
    if x <= 40:
        return x * 50 / 40
    elif x <= 80:
        return 50 + (x - 40) * 50 / 40
    elif x <= 180:
        return 100 + (x - 80) * 100 / 100
    elif x <= 280:
        return 200 + (x - 180) * 100 / 100
    elif x <= 400:
        return 300 + (x - 280) * 100 / 120
    elif x > 400:
        return 400 + (x - 400) * 100 / 120
    else:
        return 0

In [ ]:
def get_NH3_subindex(x):
    if x <= 200:
        return x * 50 / 200
    elif x <= 400:
        return 50 + (x - 200) * 50 / 200
    elif x <= 800:
        return 100 + (x - 400) * 100 / 400
    elif x <= 1200:
        return 200 + (x - 800) * 100 / 400
    elif x <= 1800:
        return 300 + (x - 1200) * 100 / 600
    elif x > 1800:
        return 400 + (x - 1800) * 100 / 600
    else:
        return 0

In [ ]:
def get_CO_subindex(x):
    if x <= 1:
        return x * 50 / 1
    elif x <= 2:
        return 50 + (x - 1) * 50 / 1
    elif x <= 10:
        return 100 + (x - 2) * 100 / 8
    elif x <= 17:
        return 200 + (x - 10) * 100 / 7
    elif x <= 34:
        return 300 + (x - 17) * 100 / 17
    elif x > 34:
        return 400 + (x - 34) * 100 / 17
    else:
        return 0

In [ ]:
def get_O3_subindex(x):
    if x <= 50:
        return x * 50 / 50
    elif x <= 100:
        return 50 + (x - 50) * 50 / 50
    elif x <= 168:
        return 100 + (x - 100) * 100 / 68
    elif x <= 208:
        return 200 + (x - 168) * 100 / 40
    elif x <= 748:
        return 300 + (x - 208) * 100 / 539
    elif x > 748:
        return 400 + (x - 400) * 100 / 539
    else:
        return 0

In [ ]:
df_index.columns

In [ ]:
df_index["o3_subIndex"] = df_index["o3_max"].apply(lambda x: get_O3_subindex(x))
df_index["co_subIndex"] = df_index["co_max"].apply(lambda x: get_CO_subindex(x))
df_index["no2_subIndex"] = df_index["no2_avg"].apply(lambda x: get_NO2_subindex(x))
df_index["so2_subIndex"] = df_index["so2_avg"].apply(lambda x: get_SO2_subindex(x))
df_index["pm2_5_subIndex"] = df_index["pm2_5_avg"].apply(lambda x: get_PM25_subindex(x))
df_index["pm10_subIndex"] = df_index["pm10_avg"].apply(lambda x: get_PM10_subindex(x))
df_index["nh3_subIndex"] = df_index["nh3_avg"].apply(lambda x: get_NH3_subindex(x))